In [30]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [31]:
# Add warehouse data
entry2_data = pd.read_csv("../entry_loc2.csv")
entry2_data = entry2_data.dropna()
entry2_data['HD entry'] = 1*(entry2_data['HD'] > 0)
entry2_data['LOW entry'] = 1*(entry2_data['LOW'] > 0)
entry2_data['log income'] = np.log(1 + entry2_data['income_per_capita'])
entry2_data['log population'] = np.log(1 + entry2_data['population'])
entry2_data['log hd warehouse'] = np.log(1 + entry2_data['hd warehouse distance'])
entry2_data['log low warehouse'] = np.log(1 + entry2_data['low warehouse distance'])
entry2_data = entry2_data[(entry2_data['population']>=10000) & (entry2_data['income_per_capita']>=15000)]
entry2_data = entry2_data[(entry2_data['lon'] <= -30) & (entry2_data['lat'] >= 25) ]
entry2_data = sm.add_constant(entry2_data)

In [46]:
class NashLogit(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)
        p = np.zeros((2, 2, n))

        for a_j in [0, 1]:
            util1 = np.dot(self.exog[:, 0:k], params[0:k]) + params[-3]*a_j
            util2 = np.dot(self.exog[:, k:2*k],params[k:2*k]) + params[-2]*a_j
            p[0, a_j, :] = 1 / (1 + np.exp(util1))
            p[1, a_j, :] = 1 / (1 + np.exp(util2))
        
        #choose lambda
        lamb = .5
        delta_pos = 1*(params[-3] >= 0)*(params[-2] >= 0)
        delta_neg = 1*(params[-3] <= 0)*(params[-2] <= 0)

        if (delta_pos == 0) and (delta_neg == 0):
            params[-2] = 0
            
        #constrain delta2 to be pos...
        
        #rho = np.clip(params[-1],-1,1)
        rho = 0
        
        # solve for probablity of nash
        prob01 = (p[0, 1, :])*(1 - p[1, 0, :])
        prob10 = (1 - p[0, 0, :])*(p[1, 1, :])
        prob00 = p[0, 0, :] * p[1, 0, :] 
        prob11 = (1 - p[0, 1, :])*(1 - p[1, 1, :])
        mult_eq =  (p[0, 1, :] - p[0, 0, :])* (p[1, 1, :] - p[1, 0, :])
        print(params[-3],params[-2],mult_eq)
        #introduce the copula
        mult_eq_rho = mult_eq*(1+rho*mult_eq)
        prob01_rho = prob01*(1+rho*prob10) - delta_neg*.5*mult_eq
        prob10_rho = prob10*(1+rho*prob01) - delta_neg*.5*mult_eq
        prob00_rho = prob00*(1+rho*prob11) - delta_pos*.5*mult_eq
        prob11_rho = prob11*(1+rho*prob00) - delta_pos*.5*mult_eq
        
            
        print(prob01_rho+prob10_rho+prob00_rho+prob11_rho)
        print('---')

        
        # compute empirical likelihood
        p00 = (1 - self.endog[:, 0])*(1 - self.endog[:, 1])
        p11 = self.endog[:, 0]*self.endog[:, 1]
        p10 = self.endog[:, 0] * (1 - self.endog[:, 1])
        p01 = (1 - self.endog[:, 0]) * self.endog[:, 1]
        

        ll = p00 * prob00_rho + p11 * prob11_rho + p01 * prob01_rho + p10 * prob10_rho
        
        return -np.log(np.maximum(ll, 1e-12))
        

    def fit(self, **kwds):
        """fit the likelihood function using the right start parameters"""
        k = int(self.exog.shape[1]/2)
        x1 = np.concatenate( (self.exog[:, 0:k], self.endog[:,1].reshape(self.endog.shape[0],1) ) ,axis=1)
        x2 = np.concatenate( (self.exog[:, k:2*k], self.endog[:,0].reshape(self.endog.shape[0],1) ),axis=1)
        params1 =  sm.Logit(self.endog[:, 0], x1).fit().params
        params2 = sm.Logit(self.endog[:, 1], x2).fit().params
        start_params = np.concatenate((params1[0:-1],params2[0:-1],[params1[-1],params2[-1], 0] ))
        #start_params = np.concatenate((params1[0:-1],params2[0:-1],[-1,-1,.8] ))

        print(start_params)
        return super(NashLogit, self).fit(start_params=start_params, **kwds)

        
        
x = entry2_data[['const','log income','log population','log hd warehouse',
                 'const','log income','log population','log low warehouse']].copy()
y = entry2_data[['HD entry','LOW entry']]

NashLogit_model = NashLogit(y,x).fit()
NashLogit_model.summary()

Optimization terminated successfully.
         Current function value: 0.516223
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.521237
         Iterations 6
[-1.09364059e+01 -5.08298059e-03  9.45608270e-01  8.33899479e-03
  4.47877722e+00 -1.15220618e+00  5.79471773e-01  5.06419108e-03
  1.33141327e+00  1.35747936e+00  0.00000000e+00]
1.3314132653557709 1.357479363551506 [0.05067017 0.05145639 0.03630503 ... 0.09896514 0.09968393 0.03271136]
[1. 1. 1. ... 1. 1. 1.]
---
1.3314132653557709 1.357479363551506 [0.03479803 0.03545917 0.02506215 ... 0.08525724 0.09856898 0.04943544]
[1. 1. 1. ... 1. 1. 1.]
---
1.3314132653557709 1.357479363551506 [0.05059102 0.05137689 0.03624597 ... 0.09892528 0.09971071 0.03278453]
[1. 1. 1. ... 1. 1. 1.]
---
1.3314132653557709 1.357479363551506 [0.06428579 0.06508033 0.04586741 ... 0.10124496 0.0889988  0.01902082]
[1. 1. 1. ... 1. 1. 1.]
---
1.3314132653557709 1.357479363551506 [0.05073349 0.05152937 0.036338

0.5049574628731075 0.9211711205572168 [0.00963161 0.00981941 0.00638657 ... 0.02399968 0.0254804  0.00938765]
[1. 1. 1. ... 1. 1. 1.]
---
0.21820586291923671 0.8617551196172457 [0.0043987  0.00448662 0.00303609 ... 0.01042021 0.0109927  0.00339397]
[1. 1. 1. ... 1. 1. 1.]
---
0.7064439907786015 1.0552765964969446 [0.01945988 0.01980113 0.01329465 ... 0.04159879 0.04122721 0.01184141]
[1. 1. 1. ... 1. 1. 1.]
---
0.631271286793768 1.0762009110585606 [0.01870134 0.01904298 0.01366363 ... 0.03900002 0.04029535 0.01127231]
[1. 1. 1. ... 1. 1. 1.]
---
0.5996928308136029 1.0374434634332248 [0.01605913 0.01635787 0.01143469 ... 0.03509061 0.03652289 0.01095124]
[1. 1. 1. ... 1. 1. 1.]
---
0.4607106454789236 0.9805124176107523 [0.01234524 0.01257145 0.00876984 ... 0.02611948 0.02646089 0.00714795]
[1. 1. 1. ... 1. 1. 1.]
---
0.3551591772853462 0.9310376521319299 [0.00897278 0.00914063 0.00637519 ... 0.01921732 0.0193877  0.00497759]
[1. 1. 1. ... 1. 1. 1.]
---
0.306758058397979 0.90898271175710

[1. 1. 1. ... 1. 1. 1.]
---
0.47984563295461113 0.9728151026128861 [0.01150819 0.01172699 0.00800858 ... 0.02608237 0.02707398 0.00817921]
[1. 1. 1. ... 1. 1. 1.]
---
0.5264910445271609 0.9851476378256275 [0.01277602 0.01301682 0.00886861 ... 0.02888117 0.02993989 0.00917733]
[1. 1. 1. ... 1. 1. 1.]
---
0.4719761590464766 0.9691903310026166 [0.0111471  0.01135994 0.00774194 ... 0.02546314 0.026517   0.00810481]
[1. 1. 1. ... 1. 1. 1.]
---
0.4722131413130808 0.9669059558662033 [0.01123811 0.01145144 0.00779323 ... 0.02549573 0.02640642 0.00797349]
[1. 1. 1. ... 1. 1. 1.]
---
0.4949490020319183 0.9768201050955877 [0.01179011 0.01201451 0.00819151 ... 0.02688427 0.02800149 0.0086089 ]
[1. 1. 1. ... 1. 1. 1.]
---
0.4558700835981212 0.9677783594483095 [0.01077298 0.01097915 0.00749463 ... 0.0245923  0.02563116 0.00781346]
[1. 1. 1. ... 1. 1. 1.]
---
0.5003603372792552 0.9761032518244166 [0.01195098 0.0121777  0.00829248 ... 0.02718093 0.02823627 0.00864335]
[1. 1. 1. ... 1. 1. 1.]
---
0.462

0.30744517305616237 1.1316090840950532 [0.00814546 0.00830925 0.00566189 ... 0.01927992 0.02019654 0.00574534]
[1. 1. 1. ... 1. 1. 1.]
---
0.26259577016377533 1.1697809124063552 [0.00709123 0.00723592 0.00492231 ... 0.01698062 0.01781703 0.00498422]
[1. 1. 1. ... 1. 1. 1.]
---
0.3057797747448826 1.1492193463221225 [0.00816703 0.00833176 0.00567532 ... 0.01942674 0.02038147 0.00579803]
[1. 1. 1. ... 1. 1. 1.]
---
0.27869087628556566 1.1660242216279113 [0.00761704 0.00777117 0.00530662 ... 0.01805036 0.01887858 0.005222  ]
[1. 1. 1. ... 1. 1. 1.]
---
0.3043306495416247 1.1757873672337393 [0.00831733 0.00848557 0.00578252 ... 0.01978779 0.02073928 0.00583424]
[1. 1. 1. ... 1. 1. 1.]
---
0.2578118611258473 1.2046954181287859 [0.00720913 0.00735644 0.00503112 ... 0.01720659 0.01806023 0.00496598]
[1. 1. 1. ... 1. 1. 1.]
---
0.22114895772293697 1.1981337996899653 [0.00614008 0.00626635 0.00427876 ... 0.01470118 0.01540613 0.00417707]
[1. 1. 1. ... 1. 1. 1.]
---
0.2149865114739561 1.201923486

0.3019689437990173 1.129360918101161 [0.0078901  0.00805163 0.00546539 ... 0.01894588 0.01980231 0.00536157]
[1. 1. 1. ... 1. 1. 1.]
---
0.32498049504541116 1.1306006680038974 [0.00853179 0.00870532 0.00590862 ... 0.02040052 0.02129887 0.00581813]
[1. 1. 1. ... 1. 1. 1.]
---
0.28083903202295923 1.160627615400164 [0.00748671 0.00764127 0.00518275 ... 0.01809066 0.0189087  0.00503768]
[1. 1. 1. ... 1. 1. 1.]
---
0.3041921591192135 1.126966838516764 [0.00791217 0.00807462 0.00548032 ... 0.01904702 0.01991355 0.0053609 ]
[1. 1. 1. ... 1. 1. 1.]
---
0.31899821071430123 1.1093795788771779 [0.008176   0.00834406 0.00565971 ... 0.01969723 0.02056698 0.00547815]
[1. 1. 1. ... 1. 1. 1.]
---
0.3171087103170175 1.1227018455290771 [0.0082192  0.00838748 0.00569355 ... 0.01977288 0.02067447 0.00558734]
[1. 1. 1. ... 1. 1. 1.]
---
0.28792221421499464 1.1433983813318949 [0.00754536 0.00770163 0.00522476 ... 0.01828844 0.01912968 0.00506   ]
[1. 1. 1. ... 1. 1. 1.]
---
0.33421490982297475 1.12581016297

...

[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.00604641 0.00618162 0.00422637 ... 0.01543892 0.01612021 0.00375442]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.00604341 0.00617856 0.00422325 ... 0.0154359  0.01611675 0.00375536]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.00604991 0.00618519 0.00422993 ... 0.01544229 0.01612283 0.00375269]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.00604209 0.0061772  0.00422128 ... 0.01543434 0.01611338 0.00375535]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.00604882 0.00618409 0.00422913 ... 0.01544138 0.0161232  0.00375356]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.006041   0.00617609 0.00422049 ... 0.01543343 0.01611375 0.00375622]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.00604787 0.00618311 0.00422757 ... 0.01544045 0.01612071 0.00375317]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343

[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.00604519 0.00618033 0.00422507 ... 0.01543761 0.0161184  0.00375462]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.00604845 0.00618371 0.00422834 ... 0.01544088 0.01612176 0.00375352]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.00604545 0.00618064 0.00422521 ... 0.01543786 0.0161183  0.00375446]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.00604545 0.00618064 0.00422521 ... 0.01543786 0.0161183  0.00375446]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.00604245 0.00617758 0.00422208 ... 0.01543484 0.01611483 0.00375539]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.00605086 0.00618617 0.0042311  ... 0.01544333 0.01612474 0.00375265]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.00604304 0.00617818 0.00422245 ... 0.01543539 0.0161153  0.00375532]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343

[1. 1. 1. ... 1. 1. 1.]
---
0.23438662368987329 1.1918973413173397 [0.00604706 0.00618228 0.00422633 ... 0.01544173 0.01612218 0.00375528]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.00604545 0.00618064 0.00422521 ... 0.01543786 0.0161183  0.00375446]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343294143601955 1.1918973413173397 [0.00604545 0.00618064 0.00422521 ... 0.01543786 0.0161183  0.00375446]
[1. 1. 1. ... 1. 1. 1.]
---
0.2342722050305177 1.1918973413173397 [0.00604385 0.006179   0.00422409 ... 0.015434   0.01611441 0.00375363]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343580190250344 1.1920428365982623 [0.00604718 0.0061824  0.0042265  ... 0.01544178 0.01612234 0.00375524]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343580190250344 1.1917518460364172 [0.00604534 0.00618052 0.00422504 ... 0.01543781 0.01611814 0.00375449]
[1. 1. 1. ... 1. 1. 1.]
---
0.2343008096953566 1.1920428365982623 [0.00604557 0.00618076 0.00422538 ... 0.01543792 0.01611845 0.00375442]
[1. 1. 1. ... 1. 1. 1.]
---
0.234

/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                  NashLogit Results                                  
=====================================================================================
Dep. Variable:     ['HD entry', 'LOW entry']   Log-Likelihood:                -3166.0
Model:                             NashLogit   AIC:                             6342.
Method:                   Maximum Likelihood   BIC:                             6372.
Date:                       Fri, 02 Jul 2021                                         
Time:                               16:00:08                                         
No. Observations:                       2952                                         
Df Residuals:                           2947                                         
Df Model:                                  4                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               -12.0249        nan        nan        nan         nan         nan
log income           -0.0140        nan        nan        nan         nan         nan
log population        1.0959        nan        nan        nan         nan         nan
log hd warehouse      0.0107        nan        nan        nan         nan         nan
const                 4.5501        nan        nan        nan         nan         nan
log income           -1.1865        nan        nan        nan         nan         nan
log population        0.6146        nan        nan        nan         nan         nan
log low warehouse     0.0016        nan        nan        nan         nan         nan
par0                  0.2343        nan        nan        nan         nan         nan
par1                  1.1919        nan        nan        nan         nan         nan
par2                 -0.0035        nan        nan        nan         nan         nan
=====================================================================================
"""